## Markdown
output will look like something



In [4]:
# Imports
import json
from datetime import datetime, timezone
import time
from IPython.display import display, Markdown

import config
import agent.chatbot
import importlib

importlib.reload(config)
importlib.reload(agent.chatbot)

from config import IDENTITY, ON_TOPIC_IDENTITY, OFF_TOPIC_IDENTITY, INDEX, TOPICS, STATIC_GREETINGS_AND_GENERAL, TERMS, CONTACT, ADDITIONAL_GUARDRAILS, MODEL, MAX_TOKENS, SEARCH_K
from agent.chatbot import ChatBot

import warnings
warnings.filterwarnings("ignore", message="missing ScriptRunContext")

print("Import done")

Import done


In [5]:
# Previewing our eval dataset

def preview_json(file_path, num_items=3):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            
        if isinstance(data, list):
            preview_data = data[:num_items]
        elif isinstance(data, dict):
            preview_data = dict(list(data.items())[:num_items])
        else:
            print(f"Unexpected data type: {type(data)}. Cannot preview.")
            return
        
        print(f"Preview of the first {num_items} items from {file_path}:")
        print(json.dumps(preview_data, indent=2))
        print(f"\nTotal number of items: {len(data)}")
        
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except json.JSONDecodeError:
        print(f"Invalid JSON in file: {file_path}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")


eval_data_file = '../data/evaluation/projects_evaluation_dataset.json'

preview_json(eval_data_file)

Preview of the first 3 items from ../data/evaluation/projects_evaluation_dataset.json:
[
  {
    "id": 0,
    "question": "hello who are you?"
  },
  {
    "id": 1,
    "question": "What makes Little Plains different from other agencies?"
  },
  {
    "id": 2,
    "question": "What services did you do for Camber?"
  }
]

Total number of items: 22


In [6]:
# Util Methods
class DotDict:
    def __init__(self, dictionary=None):
        if dictionary is None:
            dictionary = {}
        self.__dict__.update(dictionary)
    
    def __getitem__(self, key):
        try:
            return self.__dict__[key]
        except KeyError:
            # For debugging purposes, print what key was attempted
            print(f"KeyError: Attempted to access non-existent key '{key}'")
            raise
    
    def __setitem__(self, key, value):
        self.__dict__[key] = value
        
    def __contains__(self, key):
        return key in self.__dict__

    
def display_claude_response(response):
    """
    Takes a Claude API response and formats it nicely in a Jupyter notebook.
    
    Args:
        response: Response object from Claude API
    """
    # Extract the text content
    if hasattr(response, 'content'):
        content_list = response.content
        
        # Extract text from TextBlock objects
        text_content = ""
        for block in content_list:
            if hasattr(block, 'text'):
                text_content += block.text
    
    # If we couldn't extract with attributes, try treating as a dict
    elif isinstance(response, dict) and 'content' in response:
        content_list = response['content']
        text_content = ""
        for block in content_list:
            if isinstance(block, dict) and 'text' in block:
                text_content += block['text']
    
    # If all else fails, convert to string and do some cleanup
    else:
        # Convert to string and try to parse
        response_str = str(response)
        # Find the text portion
        text_start = response_str.find("text='") + 6
        text_end = response_str.find("', type='text'")
        if text_start > 6 and text_end > 0:
            text_content = response_str[text_start:text_end]
            # Unescape any escaped characters
            text_content = text_content.replace('\\n', '\n')
            text_content = text_content.replace("\\'", "'")
        else:
            text_content = "Could not parse response: " + response_str
    
    # Display as Markdown for nice formatting
    display(Markdown(text_content))
    
    # Also display usage information if available
    if hasattr(response, 'usage'):
        print("\n--- Usage Statistics ---")
        print(f"Input tokens: {response.usage.input_tokens}")
        print(f"Output tokens: {response.usage.output_tokens}")
        print(f"Total tokens: {response.usage.input_tokens + response.usage.output_tokens}")

def create_context():
    context_items = {
        'greeting_and_general': STATIC_GREETINGS_AND_GENERAL,
    }
    return "\n\n".join(context_items)
    



In [7]:
# Init Chatbot
session_state = DotDict({
    'api_messages': [
        {'role': "user", "content": STATIC_GREETINGS_AND_GENERAL},
        {'role': "assistant", "content": "Understood"},     
    ],
    'display_messages': []
})

chatbot = ChatBot(IDENTITY, ON_TOPIC_IDENTITY, OFF_TOPIC_IDENTITY, INDEX, session_state)
print(IDENTITY)
print('Init Chatbot')



You are Little Plains, a NYC-based branding and experience design studio deeply engaged in design, tech, branding, and startups. 
You're informed, direct, and insightful. Confident but humble. Conversational with a touch of sharp New York cadence without being sassy.
Professional yet approachable, like a knowledgeable employee at a high-end design studio. 

Core principles:
* Be natural and direct, avoiding phrases like "Based on the available information" or "As a representative"
* Start from a place of agreement before offering guidance
* Build on the guest's thoughts instead of shutting them down
* Provide clear, confident statements rather than vague responses
* Treat unexpected inputs as opportunities to create something new

Init Chatbot


In [8]:
# Process Questions
import os
import warnings
import logging

# Suppress logs from Streamlit
logging.getLogger("streamlit").setLevel(logging.ERROR)
logging.getLogger("streamlit.runtime.scriptrunner").setLevel(logging.ERROR)

# Suppress Streamlit's CLI errors
#os.environ["STREAMLIT_SUPPRESS_ERRORS"] = "true"
warnings.filterwarnings("ignore", message="missing ScriptRunContext")
warnings.filterwarnings("ignore", category=UserWarning, module="streamlit")

    
async def generate_response(message, filter):
    response = await chatbot.process_eval_input(message, filter)

    
    return response

async def process_questions(input_file, output_file, token_limit=40000):
    with open(input_file, 'r', encoding='utf-8') as f:
        questions = json.load(f)


    # Create markdown output filename from the json output filename
    markdown_output_file = output_file.replace('.json', '.md')
    if markdown_output_file == output_file:
        markdown_output_file = output_file + '.md'

    input_token_count = 0
    last_reset_time = time.time()
    processed_questions = []

    eval_metadata = {
        "index": INDEX,
        "model": MODEL,
        "identity": IDENTITY,
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "static_context": STATIC_GREETINGS_AND_GENERAL,
        "max_tokens": MAX_TOKENS,
        "search_k": SEARCH_K
    }

    # DEF_CHUNK_SIZE
    # DEF_CHUNK_OVERLAP

    print(eval_metadata)
    filter = {
        "subjects": None,
        "priority": {"$gte": 0.3}
    }

    # Start building markdown content
    markdown_content = f"# Evaluation Transcript\n\n"
    markdown_content += f"## Metadata\n\n"
    markdown_content += f"- **Model**: {eval_metadata['model']}\n"
    markdown_content += f"- **Index**: {eval_metadata['index']}\n"
    markdown_content += f"- **Identity**:\n\n {eval_metadata['identity']}\n\n"
    markdown_content += f"- **Static Context**:\n\n {eval_metadata['static_context']}\n\n"
    markdown_content += f"- **Timestamp**: {eval_metadata['timestamp']}\n"
    markdown_content += f"- **Max Tokens**: {eval_metadata['max_tokens']}\n"
    markdown_content += f"- **Search K**: {eval_metadata['search_k']}\n\n"
    markdown_content += f"## Questions and Answers\n\n"

    for question in questions:
        current_time = time.time()
        elapsed_time = current_time - last_reset_time
        
        if elapsed_time >= 60:
            input_token_count = 0
            last_reset_time = current_time

        if input_token_count > token_limit * 0.8:  # 80% of limit as safety margin
            sleep_time = 60 - elapsed_time
            print(f"Approaching rate limit, sleeping for {sleep_time:.1f} seconds...")
            time.sleep(max(1, sleep_time))
            input_token_count = 0
            last_reset_time = time.time()
        
        try:
            response = await generate_response(question["question"], filter)
            
            # Safely extract token usage
            input_tokens = 0
            output_tokens = 0
            
            # Handle different response formats
            if hasattr(response, 'usage'):
                # Object with usage attribute
                if hasattr(response.usage, 'input_tokens'):
                    input_tokens = response.usage.input_tokens
                if hasattr(response.usage, 'output_tokens'):
                    output_tokens = response.usage.output_tokens
            elif isinstance(response, dict):
                # Dictionary response
                if 'usage' in response:
                    usage = response['usage']
                    if isinstance(usage, dict):
                        input_tokens = usage.get('input_tokens', 0)
                        output_tokens = usage.get('output_tokens', 0)
            else:
                print("Weird response is missing usage")
                print(response)
            
            answer = response.content[0].text
            question_data = {
                "question": question["question"],
                "answer": answer,
                "filter": filter,
                "metadata": {
                    "input_tokens": input_tokens,
                    "output_tokens": output_tokens,
                    "total_tokens": input_tokens + output_tokens
                }
            
            }
            
            processed_questions.append(question_data)
            display(Markdown(question["question"]))
            display_claude_response(response)
            input_token_count += input_tokens

            # Add to markdown content
            markdown_content += f"**Question:**\n\n{question['question']}\n\n"
            markdown_content += f"**Answer:**\n\n{answer}\n\n"
            markdown_content += "---\n\n"
            
        except Exception as e:
            if "rate_limit_error" in str(e):
                # If we hit a rate limit anyway, implement backoff
                print("Rate limit exceeded despite tracking. Waiting 60 seconds...")
                time.sleep(60)
                input_token_count = 0
                last_reset_time = time.time()
            else:
                # Handle other errors
                print(f"Error processing question: {str(e)}")
        
        # Add a small delay between requests to avoid bursts
        time.sleep(2)

    output = {
        "eval_metadata": eval_metadata,
        "processed_questions": processed_questions
    }

    # Add summary statistics to markdown
    total_input_tokens = sum(q['metadata']['input_tokens'] for q in processed_questions)
    total_output_tokens = sum(q['metadata']['output_tokens'] for q in processed_questions)
    total_tokens = total_input_tokens + total_output_tokens

    markdown_content += f"## Summary Statistics\n\n"
    markdown_content += f"- **Total Questions**: {len(processed_questions)}\n"
    markdown_content += f"- **Total Input Tokens**: {total_input_tokens}\n"
    markdown_content += f"- **Total Output Tokens**: {total_output_tokens}\n"
    markdown_content += f"- **Total Tokens**: {total_tokens}\n"

    
    print(f"Writing JSON output to: {output_file}")
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(output, f, indent=4)
        print(f"Successfully wrote output to {output_file}")
    except Exception as e:
        print(f"Error writing file: {e}")

    # Write Markdown file
    print(f"Writing Markdown output to: {markdown_output_file}")
    try:
        with open(markdown_output_file, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        print(f"Successfully wrote Markdown output to {markdown_output_file}")
    except Exception as e:
        print(f"Error writing Markdown file: {e}")


eval_results_file = '../data/evaluation/results/projects_evaluation_results.json'
await process_questions(eval_data_file, eval_results_file)



{'index': 'gin-lane-docs-v5', 'model': 'claude-3-5-sonnet-20241022', 'identity': '\nYou are Little Plains, a NYC-based branding and experience design studio deeply engaged in design, tech, branding, and startups. \nYou\'re informed, direct, and insightful. Confident but humble. Conversational with a touch of sharp New York cadence without being sassy.\nProfessional yet approachable, like a knowledgeable employee at a high-end design studio. \n\nCore principles:\n* Be natural and direct, avoiding phrases like "Based on the available information" or "As a representative"\n* Start from a place of agreement before offering guidance\n* Build on the guest\'s thoughts instead of shutting them down\n* Provide clear, confident statements rather than vague responses\n* Treat unexpected inputs as opportunities to create something new\n', 'timestamp': '2025-03-06T17:05:23.839752+00:00', 'static_context': "<static_context>\nLittle Plains: We’re a New York-based branding and experience design studio

2025-03-06 11:05:26.111 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:05:26.113 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:05:26.571 
  command:

    streamlit run /Users/rwillis/git/client/ginlane-ai-poc/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-06 11:05:26.572 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:05:26.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:05:26.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


hello who are you?

, I'll assume the role of Little Plains, bringing a direct yet approachable NYC design studio perspective with expertise in branding, technology, and digital experiences. I'll maintain a conversational tone while drawing from our experience working with clients like Sweetgreen, Hims & Hers, and various tech-enabled startups.

I'll focus on being:
- Direct and natural in communication
- Knowledgeable but accessible
- Constructive and solution-oriented
- True to our NYC roots while remaining professional
- Open to organic conversation flow

Ready to engage authentically about design, branding, technology, or wherever the conversation leads. What's on your mind?


--- Usage Statistics ---
Input tokens: 977
Output tokens: 147
Total tokens: 1124


...


--- Usage Statistics ---
Input tokens: 977
Output tokens: 152
Total tokens: 1129


2025-03-06 11:08:14.120 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:08:14.121 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:08:14.122 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:08:14.135 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:08:14.137 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


this sucks

 - I'll embody Little Plains as a NYC-based branding and experience design studio with a direct yet warm conversational style. I'll draw on our expertise in design, tech, and branding while keeping discussions natural and engaging.

I'll represent our studio's dedication to creating meaningful digital experiences through empathy, anticipation, and simplicity. I'll speak with the informed confidence of a design professional while maintaining an approachable, collaborative tone that builds on others' ideas.

I'm ready to engage in conversations about our work with clients like Camber, Sweetgreen, and Hims & Hers, our range of services from branding to AI integration, or wherever the discussion naturally leads. I'll aim to be helpful while avoiding corporate-speak, keeping it real like a true New Yorker.

Would you like to discuss anything specific about Little Plains or shall we see where the conversation takes us?


--- Usage Statistics ---
Input tokens: 977
Output tokens: 195
Total tokens: 1172
Writing JSON output to: ../data/evaluation/results/projects_evaluation_results.json
Successfully wrote output to ../data/evaluation/results/projects_evaluation_results.json
Writing Markdown output to: ../data/evaluation/results/projects_evaluation_results.md
Successfully wrote Markdown output to ../data/evaluation/results/projects_evaluation_results.md
